# Usecase 1: Age prediction original set-up

This notebook can be run in the following conda environment:

```shell
conda create -n subr_r_env -c conda-forge \
    r-base=4.2.3 \
    r-randomforest \
    r-reticulate \
    r-metrics \
    r-caret \
    r-irkernel \
    python=3.10 \
    pandas \
    rpy2 \
    jupyterlab -y

conda activate subr_r_env
```


Description of modelling set-up used by original publication:           
"default parameters of R package implementation: "R package ‘randomForest’, ntree = 10,000, using default mtry of p/3 where p is the number of input 97%-identity OTUs (features)"

In [1]:
import rpy2.robjects as robjects

In [2]:
# Execute the R script
robjects.r("""source('u1_n3_original_setup.R')""")

R[write to console]: randomForest 4.7-1.1

R[write to console]: Type rfNews() to see new features/changes/bug fixes.




    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: Lade nötiges Paket: ggplot2

R[write to console]: 
Attache Paket: ‘ggplot2’


R[write to console]: Das folgende Objekt ist maskiert ‘package:randomForest’:

    margin


R[write to console]: Lade nötiges Paket: lattice

R[write to console]: 
Attache Paket: ‘caret’


R[write to console]: Die folgenden Objekte sind maskiert von ‘package:Metrics’:

    precision, recall




Training R-squared: 0.9718314 
Training RMSE: 1.098965 
Testing R-squared: 0.6878769 
Testing RMSE: 3.51052 


value,[RTYPES.NILSXP]
visible,[RTYPES.LGLSXP]
